# Kelvin-Helmholtz Instability with SPH (Continued)

By now you should have run a simulation and got a reasonable number of snapshots in the output. This notebook will now ingest those outputs and 

## What we'll do in this notebook

1. **Visualize the results** and create movies
2. **Explore different solvers/parameters** to see how they affect the instability growth

Let's get started!

In [ ]:
# Import necessary libraries
import os
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import h5py
import unyt
from unyt import msun, kpc, km, s

from swiftsimio import load
from swiftsimio.visualisation.projection import project_gas


# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Analysis and Visualization

Let's create some analysis functions that work whether or not you've run the simulation:

In [ ]:
def plot_gas_snapshot(filename, ind, title=""):
    """
    Load a SWIFT snapshot (gas only) and make 3 projection panels:
      1) Surface density (Msun/kpc^2)
      2) Mass-weighted <v_x> (km/s)
      3) Mass-weighted <|v|> in xy-plane (km/s)

    Saves to: plots/kelvin_helmholtz_{ind}.png
    """
    os.makedirs("plots", exist_ok=True)

    ds = load(filename)
    gas = ds.gas
    time_Gyr = float(ds.metadata.time.to("Gyr"))

    # Define mass-weighted fields
    vx = gas.velocities[:, 0]             # km/s
    vy = gas.velocities[:, 1]
    speed_xy = (vx**2 + vy**2)**0.5       # km/s

    gas.mass_weighted_vx = gas.masses * vx
    gas.mass_weighted_speed_xy = gas.masses * speed_xy

    # Project to 2D maps
    kwargs = dict(data=ds, resolution=512, periodic=True)
    mass_map = project_gas(project="masses", **kwargs).to_physical_value(msun / kpc**2)
    mw_vx_map = project_gas(project="mass_weighted_vx", **kwargs).to_physical_value(msun * km / (s * kpc**2))
    mw_speed_map = project_gas(project="mass_weighted_speed_xy", **kwargs).to_physical_value(msun * km / (s * kpc**2))

    # Convert to floats and do safe division
    mass = np.array(mass_map)
    vx_map = np.zeros_like(mass)
    speed_map = np.zeros_like(mass)

    mask = mass > 0
    vx_map[mask] = mw_vx_map[mask] / mass[mask]
    speed_map[mask] = mw_speed_map[mask] / mass[mask]

    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    Lx, Ly = ds.metadata.boxsize.to("kpc").value[:2]
    extent = [0.0, Lx, 0.0, Ly]

    # Panel 1: log surface density
    im1 = axes[0].imshow(
        mass,
        norm=LogNorm(),
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap="viridis",
    )
    axes[0].set_title(f"Surface density (t={time_Gyr:.3f} Gyr)")
    axes[0].set_xlabel("x [kpc]")
    axes[0].set_ylabel("y [kpc]")
    plt.colorbar(im1, ax=axes[0], label=r"$\Sigma_{\rm gas}$ [M$_\odot$/kpc$^2$]")

    # Panel 2: mass-weighted <v_x>
    im2 = axes[1].imshow(
        vx_map,
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap="RdBu_r",
    )
    axes[1].set_title(r"Mass-weighted $\langle v_x \rangle$")
    axes[1].set_xlabel("x [kpc]")
    axes[1].set_ylabel("y [kpc]")
    plt.colorbar(im2, ax=axes[1], label="km s$^{-1}$")

    # Panel 3: mass-weighted <|v|> (xy)
    im3 = axes[2].imshow(
        speed_map,
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap="plasma",
    )
    axes[2].set_title(r"Mass-weighted $\langle |v| \rangle$ (xy)")
    axes[2].set_xlabel("x [kpc]")
    axes[2].set_ylabel("y [kpc]")
    plt.colorbar(im3, ax=axes[2], label="km s$^{-1}$")

    if title:
        fig.suptitle(title, fontsize=16)

    plt.tight_layout()
    plt.savefig(f"plots/kelvin_helmholtz_{ind}.png", dpi=150, bbox_inches="tight")
    plt.show()


## 7. Loading and Analyzing Simulation Results

If the simulation ran successfully, let's load and analyze the results:

In [9]:
import glob

# Look for snapshot files
run_name = "GADGET2"
snapshot_files = sorted(glob.glob(f'../runs/{run_name}/kelvin_helmholtz_*.hdf5'))

if snapshot_files:
    print(f"Found {len(snapshot_files)} snapshot files")
    
    for i, snap in enumerate(snapshot_files):
        print(f"\nLoading snapshot {i}: {snap}")
        plot_gas_snapshot(snap, i, title=f"GADGET2: Snapshot {i}")
else:
    print("No snapshot files found. Is the path correct?")

No snapshot files found. Is the path correct?
